In [1]:
from __future__ import division, print_function

## Get the saved Keras 1.2 model

In [2]:
!pwd

/Users/avantishrikumar/Research/model_storage/deeplift/genomics


In [3]:
!ls

Convert from theano to tensorflow format.ipynb
Convert keras1.2 conv2d to keras2 conv1d model.ipynb
keras2_conv1d_record_5_model_PQzyq_modelJson.json
keras2_conv1d_record_5_model_PQzyq_modelWeights.h5
record_5_model_PQzyq_modelJson.json
record_5_model_PQzyq_modelWeights.h5
sequences.simdata.gz
test.txt.gz
test_mean_normalise_weights.ipynb
tf_order_record_5_model_PQzyq_modelJson.json
tf_order_record_5_model_PQzyq_modelWeights.h5


In [4]:
!rm tf_order_record_5_model_PQzyq_modelWeights.h5
!rm tf_order_record_5_model_PQzyq_modelJson.json
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/4143cfce7e61611d4c42984578e420cd7556c4b6/deeplift/genomics/tf_order_record_5_model_PQzyq_modelWeights.h5
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/4143cfce7e61611d4c42984578e420cd7556c4b6/deeplift/genomics/tf_order_record_5_model_PQzyq_modelJson.json

--2018-06-10 20:01:07--  https://raw.githubusercontent.com/AvantiShri/model_storage/4143cfce7e61611d4c42984578e420cd7556c4b6/deeplift/genomics/tf_order_record_5_model_PQzyq_modelWeights.h5
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150792 (147K) [application/octet-stream]
Saving to: 'tf_order_record_5_model_PQzyq_modelWeights.h5'

100%[======================================>] 150,792     --.-K/s   in 0.09s   

2018-06-10 20:01:08 (1.58 MB/s) - 'tf_order_record_5_model_PQzyq_modelWeights.h5' saved [150792/150792]

--2018-06-10 20:01:08--  https://raw.githubusercontent.com/AvantiShri/model_storage/4143cfce7e61611d4c42984578e420cd7556c4b6/deeplift/genomics/tf_order_record_5_model_PQzyq_modelJson.json
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubus

In [5]:
import h5py
import json
f = h5py.File("tf_order_record_5_model_PQzyq_modelWeights.h5")
print("keras version", f.attrs['keras_version'])
print("keras json", json.dumps(json.loads(open("tf_order_record_5_model_PQzyq_modelJson.json").read()),
                               indent=4),"\n")
print("layer weight names:", [layer_name+"/"+x
                              for layer_name in f['model_weights'].keys()
                              for x in f['model_weights'][layer_name].attrs['weight_names']],"\n")

keras version 1.2.0
keras json {
    "class_name": "Sequential", 
    "keras_version": "1.2.1", 
    "config": [
        {
            "class_name": "Convolution2D", 
            "config": {
                "W_constraint": null, 
                "b_constraint": null, 
                "name": "convolution2d_1", 
                "activity_regularizer": null, 
                "trainable": true, 
                "dim_ordering": "tf", 
                "nb_col": 11, 
                "subsample": [
                    1, 
                    1
                ], 
                "init": "glorot_uniform", 
                "bias": true, 
                "nb_filter": 50, 
                "activation": "linear", 
                "input_dtype": "float32", 
                "b_regularizer": null, 
                "W_regularizer": null, 
                "nb_row": 4, 
                "batch_input_shape": [
                    null, 
                    4, 
                    200, 
                   

/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
import keras
print ("keras version",keras.__version__)
import numpy as np

#create a keras 2 model with the same architecture
#set the weights for each layer using the hdf5
#weights file
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters=50, kernel_size=11,
                              strides=1,
                              input_shape=(200,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Conv1D(filters=50, kernel_size=11,
                              strides=1))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(units=50))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=3))
model.add(keras.layers.Activation("sigmoid"))
model.build()

#load the weights into each layer

#first convolution
model.layers[0].set_weights(
    [(np.array(f['model_weights']['convolution2d_1/convolution2d_1_W'])
              .squeeze().transpose((1,0,2))),
     np.array(f['model_weights']['convolution2d_1/convolution2d_1_b'])])
#second convolution
model.layers[2].set_weights(
    [np.array(f['model_weights']['convolution2d_2/convolution2d_2_W']).squeeze(),
     np.array(f['model_weights']['convolution2d_2/convolution2d_2_b'])])
#first dense layer
model.layers[5].set_weights(
    [np.array(f['model_weights']['dense_1/dense_1_W']),
     np.array(f['model_weights']['dense_1/dense_1_b'])])
#second dense layer
model.layers[8].set_weights(
    [np.array(f['model_weights']['dense_2/dense_2_W']),
     np.array(f['model_weights']['dense_2/dense_2_b'])])

Couldn't import dot_parser, loading of dot files will not be possible.
keras version

Using TensorFlow backend.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


 2.1.6


## Verify the accuracy

In [7]:
!rm sequences.simdata.gz
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/db919b12f750e5844402153233249bb3d24e9e9a/deeplift/genomics/sequences.simdata.gz
!rm test.txt.gz
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/9aadb769735c60eb90f7d3d896632ac749a1bdd2/deeplift/genomics/test.txt.gz

--2018-06-10 20:01:12--  https://raw.githubusercontent.com/AvantiShri/model_storage/db919b12f750e5844402153233249bb3d24e9e9a/deeplift/genomics/sequences.simdata.gz
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 629502 (615K) [application/octet-stream]
Saving to: 'sequences.simdata.gz'

100%[======================================>] 629,502     2.87MB/s   in 0.2s   

2018-06-10 20:01:13 (2.87 MB/s) - 'sequences.simdata.gz' saved [629502/629502]

--2018-06-10 20:01:13--  https://raw.githubusercontent.com/AvantiShri/model_storage/9aadb769735c60eb90f7d3d896632ac749a1bdd2/deeplift/genomics/test.txt.gz
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2287 (

In [8]:
try:
    import simdna
except ImportError:
    print("installing simdna package")
    !pip install -e "git://github.com/kundajelab/simdna.git@0.4.0#egg=simdna"
    print("\n**********************************************************")
    print("RESTART THE JUPYTER KERNEL TO PICK UP ON THE INSTALLATION!!!")
    print("************************************************************")

In [9]:
import simdna.synthetic as synthetic
reload(synthetic)
reload(synthetic.core)
import gzip
data_filename = "sequences.simdata.gz"
#read in the data in the testing set
test_ids_fh = gzip.open("test.txt.gz","rb")
ids_to_load = [x.rstrip("\n") for x in test_ids_fh]
data = synthetic.read_simdata_file(data_filename, ids_to_load=ids_to_load)

In [10]:
import numpy as np

#this model was trained on data one-hot encoded as a 2d image,
#with the row-axis being the axis for one-hot encoding.
def one_hot_encode_along_row_axis(sequence):
    #theano dim ordering, uses row axis for one-hot
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence,
                                 one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1):
        assert zeros_array.shape[0] == len(sequence)
    #zeros_array should be an array of dim 4xlen(sequence), filled with zeros.
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

onehot_data = np.array([one_hot_encode_along_row_axis(seq) for seq in data.sequences])

In [11]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(onehot_data)

print(roc_auc_score(y_score=predictions[:,0],
                    y_true=data.labels[:,0]))
print(roc_auc_score(y_score=predictions[:,1],
                    y_true=data.labels[:,1]))
print(roc_auc_score(y_score=predictions[:,2],
                    y_true=data.labels[:,2]))

0.9891813919942302
0.9884463000576744
0.9970644319550381


## Save the converted model

In [12]:
model.save_weights("keras2_conv1d_record_5_model_PQzyq_modelWeights.h5")
open("keras2_conv1d_record_5_model_PQzyq_modelJson.json",'w').write(
    json.dumps(model.get_config())
)